In [ ]:
import os
import s3fs
from pathlib import Path

from decoimpact.business.application import Application
from decoimpact.business.workflow.model_builder import ModelBuilder
from decoimpact.crosscutting.i_logger import ILogger
from decoimpact.crosscutting.logger_factory import LoggerFactory
from decoimpact.data.entities.data_access_layer import DataAccessLayer, IDataAccessLayer

In [9]:
# Create an file-system object for the s3 bucket
onxia_user_name = os.environ["GIT_USER_NAME"]
bucket_name = f"oidc-{onxia_user_name}"
S3_ENDPOINT_URL = os.environ["S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

yaml_file = "input_file.yaml"
yaml_file_path = Path(yaml_file)

In [10]:
# download the yaml file
fs.download(f"{bucket_name}/{yaml_file}", yaml_file)

In [19]:
# configure logger and data-access layer for d-ecoimpact
logger: ILogger = LoggerFactory.create_logger()
da_layer: IDataAccessLayer = DataAccessLayer(logger)
model_builder = ModelBuilder(da_layer, logger)

In [20]:
# read the input (nc) and output (nc) path from the yaml input file
model_data = da_layer.read_input_file(yaml_file_path)
nc_file = str(model_data.datasets[0].path.relative_to(os.getcwd()))
output_file_path = str(model_data.output_path)

2024-01-12 11:51:47,999: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:47,999: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:47,999: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:47,999: INFO     Creating model data based on yaml file input_file.yaml


In [21]:
# download input netCDF (UGrid) file
directory = "DFM_OUTPUT_Vietnam"
fs.download(f"{bucket_name}/{directory}/{nc_file}", nc_file)
print(f"{bucket_name}/{directory}/{nc_file}")

oidc-lmeszaros/DFM_OUTPUT_Vietnam/Vietnam_map.nc


In [22]:
# create and run d-ecoimpact application
application = Application(logger, da_layer, model_builder)
application.run(yaml_file_path)

2024-01-12 11:51:54,989: INFO     Application version: 0.1.11
2024-01-12 11:51:54,989: INFO     Application version: 0.1.11
2024-01-12 11:51:54,989: INFO     Application version: 0.1.11
2024-01-12 11:51:54,989: INFO     Application version: 0.1.11
2024-01-12 11:51:54,993: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:54,993: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:54,993: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:54,993: INFO     Creating model data based on yaml file input_file.yaml
2024-01-12 11:51:54,999: INFO     Input file version: 0.0.0
2024-01-12 11:51:54,999: INFO     Input file version: 0.0.0
2024-01-12 11:51:54,999: INFO     Input file version: 0.0.0
2024-01-12 11:51:54,999: INFO     Input file version: 0.0.0
2024-01-12 11:51:55,002: INFO     Creating rule-based model
2024-01-12 11:51:55,002: INFO     Creating rule-based model
2024-01-12 11:51:55,002: INFO     Cr

In [24]:
# upload the output file to s3
outdir = "D-EcoImpact_OUTPUT"
fs.upload(output_file_path, f"{bucket_name}/{outdir}/{output_file_path}")